### Import


In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv
import pandas as pd

C:\Users\gui07\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [5]:
!pip install langdetect

### URL

In [3]:
# base URL for the Trustpilot page

base_url = "https://www.trustpilot.com/review/www.hotels.com"
#base_url = "https://www.trustpilot.com/review/www.hotelsone.com"
#base_url = "https://www.trustpilot.com/review/www.galahotels.com"


# headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

output_file = "./scrapped_reviews/trustpilot_reviews_1.csv"
#output_file = "./scrapped_reviews/trustpilot_reviews_2.csv"
#output_file = "./scrapped_reviews/trustpilot_reviews_3.csv"

Note: you may need to restart the kernel to use updated packages.


### Functions

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import time

# Add a header to prevent blocking
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Function to scrape a single page
def scrape_page(url, page_num):
    print(f"Scraping page {page_num} from {url}")
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to load page {page_num}. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.content, "html.parser")

    # Find all review containers
    reviews = soup.find_all("article", class_="paper_paper__EGeEb")  # Update class name if structure changes

    # Store extracted reviews
    scraped_reviews = []

    for review in reviews:
        # Extract review title
        title_tag = review.find("h2", attrs={"data-service-review-title-typography": "true"})
        title = title_tag.text.strip() if title_tag else "No title"

        # Extract review body
        body_tag = review.find("p", attrs={"data-service-review-text-typography": "true"})
        body = body_tag.text.strip() if body_tag else "No body text"

        # Extract review date
        date_tag = review.find("p", attrs={"data-service-review-date-of-experience-typography": "true"})
        date = date_tag.text.strip() if date_tag else "No date"

        # Extract review score (from alt text of the image)
        score_tag = review.find("div", class_="star-rating_starRating__sdbkn")
        if score_tag:
            img_tag = score_tag.find("img")
            if img_tag and img_tag.get("alt"):
                score = img_tag["alt"].split()[1]  # Extract numeric score (e.g., "5" from "Rated 5 out of 5 stars")
            else:
                score = "No score"
        else:
            score = "No score"

        # Append the review data
        scraped_reviews.append({
            "title": title,
            "body": body,
            "date": date,
            "score": score
        })

    return scraped_reviews


# Main function to scrape multiple URLs
def scrape_multiple_urls(url_list, max_pages, output_file):
    all_reviews = []

    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["source_url", "title", "body", "date", "score"])
        writer.writeheader()

        for url in url_list:
            for page_num in range(1, max_pages + 1):
                page_url = f"{url}?page={page_num}"
                reviews = scrape_page(page_url, page_num)

                # Add source URL to each review
                for review in reviews:
                    review["source_url"] = url

                all_reviews.extend(reviews)

                # Write to CSV after every page
                writer.writerows(reviews)

                if not reviews:
                    print(f"No more reviews found for {url}. Stopping at page {page_num}.")
                    break

                time.sleep(2)

    print(f"Scraping complete. {len(all_reviews)} reviews saved to {output_file}.")
    return all_reviews

# Example usage
url_list = [
    "https://www.trustpilot.com/review/www.hotels.com",
    "https://www.trustpilot.com/review/www.booking.com",
    "https://www.trustpilot.com/review/www.airbnb.com",
    "https://www.trustpilot.com/review/www.hotels.com",
    "https://www.trustpilot.com/review/www.hostelworld.com"
    
]
output_file = "reviews_multiple_urls.csv"
max_pages = 10

scrape_multiple_urls(url_list, max_pages, output_file)



Scraping page 1 from https://www.trustpilot.com/review/www.hotels.com?page=1
Scraping page 2 from https://www.trustpilot.com/review/www.hotels.com?page=2
Scraping page 3 from https://www.trustpilot.com/review/www.hotels.com?page=3
Scraping page 4 from https://www.trustpilot.com/review/www.hotels.com?page=4
Scraping page 5 from https://www.trustpilot.com/review/www.hotels.com?page=5
Scraping page 6 from https://www.trustpilot.com/review/www.hotels.com?page=6
Scraping page 7 from https://www.trustpilot.com/review/www.hotels.com?page=7
Scraping page 8 from https://www.trustpilot.com/review/www.hotels.com?page=8
Scraping page 9 from https://www.trustpilot.com/review/www.hotels.com?page=9
Scraping page 10 from https://www.trustpilot.com/review/www.hotels.com?page=10
Scraping page 1 from https://www.trustpilot.com/review/www.booking.com?page=1
Scraping page 2 from https://www.trustpilot.com/review/www.booking.com?page=2
Scraping page 3 from https://www.trustpilot.com/review/www.booking.com?p

[{'title': 'I wish I could leave a zero rating.',
  'body': "I wish I could leave a zero rating.Absolutely terrible customer services. I booked a hotel incl £70 tesco voucher. The hotel cancelled my booking.Tesco say its nothing to do with them.I've been emailing booking.com since 3rd January they keep saying they are waiting for a response from Tesco.A terrible company stay clear",
  'date': 'Date of experience: January 17, 2025',
  'score': '1',
  'source_url': 'https://www.trustpilot.com/review/www.hotels.com'},
 {'title': 'Careful if you speak with some agents for any issues.',
  'body': 'First, I needed a reply from the hotel side, but I didn’t get any response, so I contacted Hotel.com customer service via live chat.Initially, I spoke with Andrea Alexandra to get a reply to my request. She tried calling the hotel, but was unable to connect. She then mentioned she would contact me when she received a reply. However, I didn’t get any reply even after more than two weeks, so I conta

In [97]:
pip install selenium

WebDriverException: Message: unknown error: cannot find Chrome binary
Stacktrace:
Backtrace:
	GetHandleVerifier [0x008DA813+48355]
	(No symbol) [0x0086C4B1]
	(No symbol) [0x00775358]
	(No symbol) [0x00791A9E]
	(No symbol) [0x00790579]
	(No symbol) [0x007C0C55]
	(No symbol) [0x007C093C]
	(No symbol) [0x007BA536]
	(No symbol) [0x007982DC]
	(No symbol) [0x007993DD]
	GetHandleVerifier [0x00B3AABD+2539405]
	GetHandleVerifier [0x00B7A78F+2800735]
	GetHandleVerifier [0x00B7456C+2775612]
	GetHandleVerifier [0x009651E0+616112]
	(No symbol) [0x00875F8C]
	(No symbol) [0x00872328]
	(No symbol) [0x0087240B]
	(No symbol) [0x00864FF7]
	BaseThreadInitThunk [0x778AFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77B6809E+286]
	RtlGetAppContainerNamedObjectPath [0x77B6806E+238]


In [95]:
pip install webdriver-manager

  Obtaining dependency information for webdriver-manager from https://files.pythonhosted.org/packages/b5/b5/3bd0b038d80950ec13e6a2c8d03ed8354867dc60064b172f2f4ffac8afbe/webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata
Note: you may need to restart the kernel to use updated packages.


### RSE multilanguage

In [7]:
import csv
from langdetect import detect

# Define RSE related keywords for multiple languages
keywords = {
    "en": {
        "positive": ["eco-friendly", "sustainable", "green", "energy", "fair", "wages", "renewable", "recycling", "ethical", "carbon", "neutral", "clean", "organic", "friendly", "low", "emissions", "initiatives", "diversity", "community", "local", "efficiency", "responsible"],
        "negative": ["greenwashing", "unethical", "unsustainable", "wasteful", "exploitation", "pollution", "bad", "conditions", "high", "emissions", "deforestation", "toxic", "violations", "child", "labor", "lack", "transparency", "damage"]
    },
    "fr": {
        "positive": ["éco", "responsable", "durable", "énergie", "verte", "salaires", "équitables", "renouvelable", "recyclage", "éthique", "carbone", "neutre", "propre", "bio", "respectueux", "environnement", "basses", "émissions", "initiatives", "diversité", "communauté", "locale", "efficacité", "tourisme"],
        "negative": ["écoblanchiment", "non éthique", "durable", "gaspillage", "exploitation", "pollution", "mauvaises", "conditions", "élevées", "émissions", "déforestation", "toxique", "violations", "droits", "enfants", "manque", "transparence", "dégradations"]
    },
    "es": {
        "positive": ["ecológico", "sostenible", "energía", "verde", "salarios", "justos", "renovable", "reciclaje", "ético", "carbono", "neutral", "limpia", "orgánico", "respetuoso", "medio", "ambiente", "bajas", "emisiones", "iniciativas", "diversidad", "comunidad", "local", "eficiencia", "turismo"],
        "negative": ["lavado", "verde", "no ético", "sostenible", "desperdicio", "explotación", "contaminación", "malas", "condiciones", "altas", "emisiones", "deforestación", "tóxico", "violaciones", "derechos", "infantil", "falta", "transparencia", "daños"]
    },
    "de": {
        "positive": ["umwelt", "freundlich", "nachhaltig", "grün", "energie", "faire", "löhne", "erneuerbar", "recycling", "ethisch", "klima", "neutral", "sauber", "bio", "freundlich", "niedrige", "emissionen", "initiativen", "vielfalt", "gemeinschaft", "lokal", "effizienz", "tourismus"],
        "negative": ["greenwashing", "unethisch",  "nachhaltig", "verschwendung", "ausbeutung", "verschmutzung", "schlechte", "bedingungen", "hohe", "emissionen", "abholzung", "giftig", "menschen", "rechte", "kinderarbeit", "mangel", "transparenz", "schäden"]
    },
    "pt": {
        "positive": ["ecológico", "sustentável", "energia", "verde", "salários", "justos", "renovável", "reciclagem", "ético", "carbono", "neutro", "limpa", "orgânico", "amigável", "meio", "ambiente", "baixas", "emissões", "iniciativas", "diversidade", "comunidade", "local", "eficiência", "turismo"],
        "negative": ["greenwashing", "antiético", "sustentável", "desperdício", "exploração", "poluição", "más", "condições", "altas", "emissões", "desmatamento", "tóxico", "violaciones", "direitos", "infantil", "falta", "transparência", "danos"]
    },
    "it": {
        "positive": ["ecologico", "sostenibile", "energia", "verde", "salari", "equi", "rinnovabile", "riciclaggio", "etico", "carbonio", "neutro", "pulita", "bio", "rispettoso", "ambiente", "basse", "emissioni", "iniziative", "diversità", "comunità", "locale", "efficienza", "turismo"],
        "negative": ["greenwashing", "non etico", "sostenibile", "spreco", "sfruttamento", "inquinamento", "cattive", "condizioni", "alte", "emissioni", "deforestazione", "tossico", "violazioni", "diritti", "minori", "mancanza", "trasparenza", "danni"]
    }
}


input_file = "./scrapped_reviews/trustpilot_reviews_1.csv"  # Input CSV file with scraped reviews
output_file = "./test_classification/trustpilot_rse_classified_reviews_multilingual_1.csv"  # Output CSV file with classifications

# Function to classify reviews based on RSE keywords
def classify_review(text, language):
    text_lower = text.lower()  # Convert to lowercase for case insensitive matching
    lang_keywords = keywords.get(language, None)  # Get keywords for the detected language

    if lang_keywords:
        if any(keyword in text_lower for keyword in lang_keywords["positive"]):
            return "Positive"

        if any(keyword in text_lower for keyword in lang_keywords["negative"]):
            return "Negative"

    return "Neutral"

# Read reviews from the input file and classify them
classified_reviews = []

with open(input_file, mode="r", encoding="utf-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        review_body = row["body"]  # The review text column

        # Detect the language of the review
        try:
            language = detect(review_body)
        except:
            language = "unknown"

        # Classify the review based on the detected language
        classification = classify_review(review_body, language)
        row["language"] = language  # Add detected language to the row
        row["classification"] = classification  # Add classification to the row
        classified_reviews.append(row)

# Write the classified reviews to a new CSV file
with open(output_file, mode="w", newline="", encoding="utf-8") as file:
    fieldnames = ["title", "body", "date", "language", "classification"]  # Include language and classification columns
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(classified_reviews)

print(f"Multilingual RSE classification complete. Results saved to {output_file}.")


Multilingual RSE classification complete. Results saved to ./test_classification/trustpilot_rse_classified_reviews_multilingual_1.csv.


In [9]:
df = pd.read_csv("./test_classification/trustpilot_rse_classified_reviews_multilingual_1.csv")
df1 = df[df["classification"]=="Negative"]
df1["body"]

5       I found this site using ChatGPT (I've tried to...
14      Hotel didn't guarantee my reservation.  They d...
25      I found this site using ChatGPT (I've tried to...
34      Hotel didn't guarantee my reservation.  They d...
45      I found this site using ChatGPT (I've tried to...
                              ...                        
4954    Hotel didn't guarantee my reservation.  They d...
4965    I found this site using ChatGPT (I've tried to...
4974    Hotel didn't guarantee my reservation.  They d...
4985    I found this site using ChatGPT (I've tried to...
4994    Hotel didn't guarantee my reservation.  They d...
Name: body, Length: 500, dtype: object

In [11]:
df = pd.read_csv("./test_classification/trustpilot_rse_classified_reviews_multilingual_1.csv")
df1 = df[df["classification"]=="Positive"]
df1["body"]

3       This was a great hotel. very comfortable and p...
4       Friendly staff, good location, clean and spaci...
12      Initially changed to HRS to support a local bu...
16      This site has often special offers for high qu...
23      This was a great hotel. very comfortable and p...
                              ...                        
4976    This site has often special offers for high qu...
4983    This was a great hotel. very comfortable and p...
4984    Friendly staff, good location, clean and spaci...
4992    Initially changed to HRS to support a local bu...
4996    This site has often special offers for high qu...
Name: body, Length: 1000, dtype: object